<a href="https://colab.research.google.com/github/hursoo/big_k-modern_1/blob/main/gb_031_make_2_gram(big1_251)_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.개요
- DMR 적용해서 "개벽" 논조변화 분석
- 입력 데이터: 'gb_data_2.1.xlsx'(어순 많이 교정 + 필자 그룹 분류 중 사회주의 필진 보충 + 0,1,2,3의 네 그룹으로 단순화)
- 코딩: 2-gram을 활용
    - 2-gram 생성 후 여기에 참여한 좌우단어는 각각 삭제

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 경로 지정

file_path = '/content/drive/MyDrive/_big1_25-1_code/'

In [ ]:
!pip install tomotopy

In [ ]:
import sys
import os, re
import pandas as pd
import numpy as np
import random
import warnings

from tomotopy import DMRModel
from tomotopy import TermWeight
from tomotopy import utils
import tomotopy as tp

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호가 정상 표시되도록 설정

# 2.데이터 정보 연결
- 사전 전처리와 구조화된 DB(관계형)
- 문장, 논설, 호, 구간 정보
- 5개 문장을 1개 문서(doc)로 통합
- 2-gram 반영

In [ ]:
# db의 각 탭의 필요 정보를 불러와서 변수에 각각 할당
sent = pd.read_excel(file_path + 'data/gb_data_2.1.xlsx', sheet_name = 'sent') # 문장별 전처리한 결과물
r = pd.read_excel(file_path + 'data/gb_data_2.1.xlsx', sheet_name = 'ron').drop('ho_no', axis=1) # 논설 정보
ho = pd.read_excel(file_path + 'data/gb_data_2.1.xlsx', sheet_name = 'ho').drop('grid', axis=1) # 호별 정보
writer = pd.read_excel(file_path + 'data/gb_data_2.1.xlsx', sheet_name = 'writer_new') # 필자 정보

## 2.1.5개 문장을 통합
- 토픽모델링에 의한 토픽 추출에는 문장 단위보다 긴 텍스트가 적절하다고 판단.
- 편의상 5문장을 통합하여 1개의 문서로 만듦.
- 34,030개 -> 6,802개

In [ ]:
# 통합 함수
def integrate_sentences(df):
    result = []
    doc_id_counter = 1

    # 기사 단위로 처리 (r_no, ho_no 기준 그룹)
    for (r_no, ho_no), group in df.groupby(['r_no', 'ho_no']):
        sent_raw_list = group['sent_raw'].tolist()
        sent_split_list = group['sent_split'].tolist()

        integrated_raw = []
        integrated_split = []
        temp_raw = []
        temp_split = []

        for raw, split in zip(sent_raw_list, sent_split_list):
            temp_raw.append(raw)
            temp_split.append(split)

            # 5개씩 묶기
            if len(temp_raw) == 5:
                integrated_raw.append(temp_raw)
                integrated_split.append(temp_split)
                temp_raw = []
                temp_split = []

        # 자투리 문장 처리
        if len(temp_raw) > 0:
            if len(temp_raw) <= 2 and len(integrated_raw) > 0:  # 자투리 2개 이하
                integrated_raw[-1].extend(temp_raw)
                integrated_split[-1].extend(temp_split)
            else:  # 자투리 3개 이상
                integrated_raw.append(temp_raw)
                integrated_split.append(temp_split)

        # 결과 저장
        for raw_group, split_group in zip(integrated_raw, integrated_split):
            result.append({
                'doc_id': doc_id_counter,
                'doc_raw': " ".join(raw_group),
                'doc_split': " ".join(split_group),
                'r_no': r_no,
                'ho_no': ho_no
            })
            doc_id_counter += 1

    return pd.DataFrame(result)

# 함수 실행
result_df = integrate_sentences(sent)

# 결과 출력
result_df

,doc_id,doc_raw,doc_split,r_no,ho_no
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...,1,1
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...,1,1
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...,2,1
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...,2,1
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...,2,1
...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...,334,72
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제 연맹 국제 연맹 원인 경제 機械 캐나다 산업 북미 자본 점령 부분...,334,72
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...,334,72


## 2.2.2-gram 생성, 삽입
- 목적: 전처리시 잘게 나눈 단어들 중 복합어 등으로 조합되면 문맥 의미를 더 잘 보이는 경우가 있음. 이런 정보를 반영하고자 함.
- 예: 사회 + 主義 -> 사회_主義

### 2.2.1.생성(빈도5이상)
- 2-gram 단어를 생성해서 해당 두 단어 사이에 위치 시킴
- 방만한 단어 방지 위해 2-gram 단어로 빈도 5 이하는 제거

In [ ]:
import pandas as pd
from nltk import ngrams
from collections import Counter

# 데이터프레임
df = result_df
# 데이터프레임 복사본 생성 (권장)
df = df.copy()

# Step 1: 2-gram 생성 및 빈도 계산
def generate_bigrams(text):
    words = text.split()
    return ["_".join(bigram) for bigram in ngrams(words, 2)]  # 단어 사이에 _ 추가

# 모든 바이그램 수집
all_bigrams = []
df["doc_split"].apply(lambda x: all_bigrams.extend(generate_bigrams(x)))

# 바이그램 빈도 계산
bigram_counts = Counter(all_bigrams)

# 빈도 기준 절삭
threshold = 5  # 예시: 빈도가 5 이하인 경우 제거
filtered_bigrams = {k: v for k, v in bigram_counts.items() if v > threshold}

# Step 2: 절삭 후 남은 2-gram 삽입
def insert_bigrams(row, filtered_bigrams):
    words = row.split()
    new_words = []
    for i in range(len(words) - 1):
        bigram = "_".join((words[i], words[i + 1]))  # 단어 사이에 _ 추가
        new_words.append(words[i])
        if bigram in filtered_bigrams:
            # Bigram을 단일 문자열로 삽입
            new_words.append(bigram)
    new_words.append(words[-1])  # 마지막 단어 추가
    return " ".join(new_words)

# 데이터프레임에 새로운 열 추가
df["doc_split_updated"] = df["doc_split"].apply(lambda x: insert_bigrams(x, filtered_bigrams))
df

,doc_id,doc_raw,doc_split,r_no,ho_no,doc_split_updated
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...,1,1,창간 辭 강자 강자_약자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리_소리 소리...
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...,1,1,哲人 다수 다수_인민 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 다...
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...,2,1,세계 세계_사람 사람 야수 사람 사람_진화 진화 진화_진화 진화 지식 진화 도덕 동...
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...,2,1,사람 사람_세계 세계 세계_사람 사람 사람_세계 세계 세계_세계 세계 세계_사람 사...
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...,2,1,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하_천하 천하 理解 오...
...,...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...,334,72,저축 필요 비용 미국 교과 교과_書 書 실상 미국 富豪 캐나다 영국 영국_미국 미국...
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제 연맹 국제 연맹 원인 경제 機械 캐나다 산업 북미 자본 점령 부분...,334,72,캐나다 미국 국제 국제_연맹 연맹 연맹_국제 국제 국제_연맹 연맹 원인 경제 機械 ...
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...,334,72,영국 영국_미국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본...


In [ ]:
# 필요한 열만 남김
df1 = df[['doc_id', 'doc_raw', 'doc_split_updated', 'r_no', 'ho_no']]
df1

,doc_id,doc_raw,doc_split_updated,r_no,ho_no
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 강자_약자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리_소리 소리...,1,1
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 다수_인민 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 다...,1,1
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 세계_사람 사람 야수 사람 사람_진화 진화 진화_진화 진화 지식 진화 도덕 동...,2,1
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 사람_세계 세계 세계_사람 사람 사람_세계 세계 세계_세계 세계 세계_사람 사...,2,1
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하_천하 천하 理解 오...,2,1
...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 교과_書 書 실상 미국 富豪 캐나다 영국 영국_미국 미국...,334,72
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제 국제_연맹 연맹 연맹_국제 국제 국제_연맹 연맹 원인 경제 機械 ...,334,72
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 영국_미국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본...,334,72


### 2.2.2.선별(빈도30이상 등)
- 동어 반복, 어색한 조합 등은 제외
- 빈도 30회 이상만 남기고 나머지는 삭제
- 이 과정에서 기존의 복합어(언더바로 수작업 연결한 것)도 절삭됨.

In [ ]:
# 2-gram 빈도 파악

import pandas as pd
from collections import Counter

# 언더바 포함 단어의 빈도 계산 함수
def count_underscored_bigrams(column):
    words = []
    for row in column:
        words.extend([word for word in row.split() if "_" in word])  # 언더바 포함 단어만 추출
    return Counter(words)

# 언더바가 포함된 단어의 빈도 계산
bigram_frequencies = count_underscored_bigrams(df1["doc_split_updated"])

# 빈도를 데이터프레임으로 변환 및 정렬
bigram_freq_df = pd.DataFrame(bigram_frequencies.items(), columns=["bigram", "frequency"])
bigram_freq_df = bigram_freq_df.sort_values(by="frequency", ascending=False).reset_index(drop=True)

# 결과 출력
bigram_freq_df

,bigram,frequency
0,朝鮮_人,981
1,노동_者,489
2,사회_主義,432
3,자본_主義,415
4,소작_人,380
...,...,...
4580,민주_黨,1
4581,갑오_경장,1
4582,천도_敎_청년_黨,1
4583,朝鮮_농민_社,1


In [ ]:
bigram_freq_df.tail()

,bigram,frequency
4580,민주_黨,1
4581,갑오_경장,1
4582,천도_敎_청년_黨,1
4583,朝鮮_농민_社,1
4584,노동_總_동맹,1


In [ ]:
# 1-60위 까지

print(bigram_freq_df[:60]['bigram'].tolist())

['朝鮮_人', '노동_者', '사회_主義', '자본_主義', '소작_人', '朝鮮_민족', '朝鮮_사람', '主義_者', '사람_사람', '오늘_날', '자본_家', '무산_계급', '사람_性', '사회_운동', '기독_敎', '朝鮮_사회', '帝國_主義', '민족_민족', '中産_계급', '천도_敎', '無_정부', '농업_노동', '정부_主義', '사회_생활', '단체_생활', '일본_人', '계급_意識', '식민_地', '主義_사회', '지식_계급', '사회_사회', '노동_계급', '사회_문제', '계급_계급', '自己_自己', '민족_主義', '노동_문제', '인류_사회', '민족_개조', '사회_제도', '공산_主義', '문화_운동', '무산_者', '指導_者', '사람_自己', '일반_민중', '사회_조직', '인생_觀', '朝鮮_朝鮮', '공산_黨', '민족_생활', '문제_해결', '민주_主義', '人_생활', '외국_人', '금일_朝鮮', '생산_力', '민족_감정', '일반_사회', '人_朝鮮']


In [ ]:
# 1-60위 중 일부 어색한 조합 제외한 결과: 예) 어색한 조합 - '主義_사회', '朝鮮_朝鮮' 등

top_2gram = ['朝鮮_人', '노동_者', '사회_主義', '자본_主義', '소작_人', '朝鮮_민족', '朝鮮_사람', '主義_者', '자본_家', '무산_계급', '사람_性', '사회_운동',
 '기독_敎', '朝鮮_사회', '帝國_主義', '中産_계급', '천도_敎', '無_정부', '농업_노동', '정부_主義', '사회_생활', '단체_생활', '일본_人', '계급_意識',
 '식민_地', '지식_계급', '노동_계급', '사회_문제', '민족_主義', '노동_문제', '인류_사회', '민족_개조', '사회_제도', '공산_主義', '문화_운동', '무산_者',
 '指導_者', '사람_自己', '일반_민중', '사회_조직', '인생_觀', '공산_黨', '민족_생활', '문제_해결', '민주_主義', '외국_人', '생산_力', '민족_감정',
 '일반_사회', ]

2-gram의 범위를 조금 더 넓히기로 함
- 빈도를 50이상, 나아가 30 이상으로 확장 (아래 두 개의 셀 참조)

In [ ]:
# 선별된 단어 리스트: 2-gram 단어 중 빈도 "50" 이상 중에서 어색한 조합은 제외한 것
selected_bigrams = [
    "자유_평등", "어린_이", "마르크스_主義", "민족_性", "新_문화", "지주_소작", "자본_계급",
    "계급_투쟁", "사회_현상", "정치_경제", "현대_문명", "생활_難", "중심_인물", "지배_계급",
    "현대_사회", "생활_조건", "경제_정책", "인류_생활", "사상_家", "人道_正義", "朝鮮_민중",
    "노동_운동", "농촌_문제", "자연_主義", "지배_者", "도착_點", "현대_人", "약소_민족",
    "물산_장려", "프랑스_혁명", "專門_家", "생활_표준", "개인_主義", "계급_운동", "유산_者",
    "자유_主義", "사상_문화", "당국_者", "정치_운동", "문화_생활", "중심_세력", "정치_家"
]

In [ ]:
# 50미만 30이상 추가
a = '종교_家 태평_洋 新_사회 不_완전 唯物_論 출발_點 하나_님 正_반대 不_합리 유럽_대전 민족_체면 사회_진화 사회_계급 생활_費 서양_人 인간_생활'
b = '소작_제도 사상_혁명 민족_운동 朝鮮_운동 중류_계급 생활_양식 경제_문제 생산_者 생산_관계 인류_구제 압박_민족 계급_지배 자연_과학 唯物_史觀 자본_제도 토지_소유'
c = '중추_계급 선교_師 婦人_문제 理想_主義 사회_개조 생활_향상 경제_생활 경제_조직 봉건_제도 볼세비키_主義 사회_경제 인간_사회 식민_정책 국가_민족'
d = '소작_料 사회_봉공 야만_人 人道_主義 무산_청년 예술_家 문화_건설 계급_대립 大_지주 사회_學 생활_문화 인류_主義 민중_운동 민중_운동 사회_생산 청년_운동'
e = '세계_개조 사회_黨 被_압박 부르주아_문화 문예_부흥 사회_性 부르주아_사회 군국_主義 소작_운동 유산_계급 소작_農 自作_農 가족_제도 민족_사회 朝鮮_농촌'
f = '경제_운동 공업_노동 實業_家 생활_문제 특권_계급 생활_維持 민족_중심 체면_維持 혁명_운동 동양_척식_회사 원동_力 정신_물질 보통_학교 러시아_혁명'
g = '총독_府 교육_문제 私有_재산 국제_연맹 被_정복 소비_者 청년_會 사회_혁명 종교_신앙 사회_건설 사회_사상 新_사상'

In [ ]:
# 문자열 합치기
combined_text = f"{a} {b} {c} {d} {e} {f} {g}"

# 공백으로 구분하여 리스트로 변환
word_list = combined_text.split()

# 결과 확인
print(word_list)

['종교_家', '태평_洋', '新_사회', '不_완전', '唯物_論', '출발_點', '하나_님', '正_반대', '不_합리', '유럽_대전', '민족_체면', '사회_진화', '사회_계급', '생활_費', '서양_人', '인간_생활', '소작_제도', '사상_혁명', '민족_운동', '朝鮮_운동', '중류_계급', '생활_양식', '경제_문제', '생산_者', '생산_관계', '인류_구제', '압박_민족', '계급_지배', '자연_과학', '唯物_史觀', '자본_제도', '토지_소유', '중추_계급', '선교_師', '婦人_문제', '理想_主義', '사회_개조', '생활_향상', '경제_생활', '경제_조직', '봉건_제도', '볼세비키_主義', '사회_경제', '인간_사회', '식민_정책', '국가_민족', '소작_料', '사회_봉공', '야만_人', '人道_主義', '무산_청년', '예술_家', '문화_건설', '계급_대립', '大_지주', '사회_學', '생활_문화', '인류_主義', '민중_운동', '민중_운동', '사회_생산', '청년_운동', '세계_개조', '사회_黨', '被_압박', '부르주아_문화', '문예_부흥', '사회_性', '부르주아_사회', '군국_主義', '소작_운동', '유산_계급', '소작_農', '自作_農', '가족_제도', '민족_사회', '朝鮮_농촌', '경제_운동', '공업_노동', '實業_家', '생활_문제', '특권_계급', '생활_維持', '민족_중심', '체면_維持', '혁명_운동', '동양_척식_회사', '원동_力', '정신_물질', '보통_학교', '러시아_혁명', '총독_府', '교육_문제', '私有_재산', '국제_연맹', '被_정복', '소비_者', '청년_會', '사회_혁명', '종교_신앙', '사회_건설', '사회_사상', '新_사상']


In [ ]:
# 2-gram 단어 모두 합침: 빈도 상위 60개 + 빈도 30회 이상 단어 = 총 194개 2gram
ttl_bigram = top_2gram + selected_bigrams + word_list
print(len(ttl_bigram))
print(ttl_bigram)

194
['朝鮮_人', '노동_者', '사회_主義', '자본_主義', '소작_人', '朝鮮_민족', '朝鮮_사람', '主義_者', '자본_家', '무산_계급', '사람_性', '사회_운동', '기독_敎', '朝鮮_사회', '帝國_主義', '中産_계급', '천도_敎', '無_정부', '농업_노동', '정부_主義', '사회_생활', '단체_생활', '일본_人', '계급_意識', '식민_地', '지식_계급', '노동_계급', '사회_문제', '민족_主義', '노동_문제', '인류_사회', '민족_개조', '사회_제도', '공산_主義', '문화_운동', '무산_者', '指導_者', '사람_自己', '일반_민중', '사회_조직', '인생_觀', '공산_黨', '민족_생활', '문제_해결', '민주_主義', '외국_人', '생산_力', '민족_감정', '일반_사회', '자유_평등', '어린_이', '마르크스_主義', '민족_性', '新_문화', '지주_소작', '자본_계급', '계급_투쟁', '사회_현상', '정치_경제', '현대_문명', '생활_難', '중심_인물', '지배_계급', '현대_사회', '생활_조건', '경제_정책', '인류_생활', '사상_家', '人道_正義', '朝鮮_민중', '노동_운동', '농촌_문제', '자연_主義', '지배_者', '도착_點', '현대_人', '약소_민족', '물산_장려', '프랑스_혁명', '專門_家', '생활_표준', '개인_主義', '계급_운동', '유산_者', '자유_主義', '사상_문화', '당국_者', '정치_운동', '문화_생활', '중심_세력', '정치_家', '종교_家', '태평_洋', '新_사회', '不_완전', '唯物_論', '출발_點', '하나_님', '正_반대', '不_합리', '유럽_대전', '민족_체면', '사회_진화', '사회_계급', '생활_費', '서양_人', '인간_생활', '소작_제도', '사상_혁명', '민족_운동', '朝鮮_운동', '중류_계급', '생활_양식', '경제_문제', '생산_者'

In [ ]:
# 2-gram 중에서 선별된 단어만 남기는 함수
def filter_selected_bigrams_with_all_unigrams(text, selected_bigrams):
    words = text.split()
    filtered_words = [
        word for word in words
        if "_" not in word or word in selected_bigrams  # 1-gram은 모두 포함, 2-gram은 선별된 리스트에 있는 것만
    ]
    return " ".join(filtered_words)

# 필터링 적용
df1 = df1.copy()
df1["doc_split_filtered"] = df1["doc_split_updated"].apply(
    lambda x: filter_selected_bigrams_with_all_unigrams(x, ttl_bigram)
)

# 결과 확인
df1

,doc_id,doc_raw,doc_split_updated,r_no,ho_no,doc_split_filtered
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 강자_약자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리_소리 소리...,1,1,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 다수_인민 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 다...,1,1,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 세계_사람 사람 야수 사람 사람_진화 진화 진화_진화 진화 지식 진화 도덕 동...,2,1,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 사람_세계 세계 세계_사람 사람 사람_세계 세계 세계_세계 세계 세계_사람 사...,2,1,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하_천하 천하 理解 오...,2,1,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...
...,...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 교과_書 書 실상 미국 富豪 캐나다 영국 영국_미국 미국...,334,72,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제 국제_연맹 연맹 연맹_국제 국제 국제_연맹 연맹 원인 경제 機械 ...,334,72,캐나다 미국 국제 국제_연맹 연맹 국제 국제_연맹 연맹 원인 경제 機械 캐나다 산업...
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 영국_미국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본...,334,72,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...


### 2.2.3.정리(앞뒤 단어 제거)
- 2-gram을 남길 때 그 앞 뒤의 단어로 이 2-gram에 참여한 단어는 제거
- 단어별 중복 계산 방지 위함

In [ ]:
# 남은 2gram 앞뒤 단어 삭제 (단어의 빈도 계산시 중복 방지 위해)

# 1. 문장에서 앞에서부터 '_'가 있는 단어(2gram)를 찾는다.
# 2. 이 단어를 _ 기준으로 쪼갠다.
# 3. 쪼갠 뒤의 왼쪽 단어를 이 2gram 바로 앞 단어와 비교하여 동일하면 앞 단어를 삭제한다.
# 4. 쪼갠 뒤의 오른쪽 단어를 이 2gram 바로 뒤 단어와 비교하여 동일하면 뒤 단어를 삭제한다.
# 5. 이 작업을 동일 문장의 다음 2gram에 대해서도 실행한다.

def remove_adjacent_words_with_bigram(row):
    words = row.split()
    new_words = words[:]  # 원본을 복사해서 수정

    i = 0
    while i < len(new_words):
        # 1. '_'가 있는 단어 찾기
        if "_" in new_words[i]:
            # '_'가 1개만 포함된 경우만 처리 (2gram만)
            if new_words[i].count("_") == 1:
                # 2. '_' 기준으로 쪼갬
                left_word, right_word = new_words[i].split("_")

                # 3. 쪼갠 뒤 왼쪽 단어와 2gram 바로 앞 단어 비교
                if i > 0 and new_words[i - 1] == left_word:
                    new_words.pop(i - 1)  # 앞 단어 삭제
                    i -= 1  # 삭제했으니 인덱스 이동

                # 4. 쪼갠 뒤 오른쪽 단어와 2gram 바로 뒤 단어 비교
                if i < len(new_words) - 1 and new_words[i + 1] == right_word:
                    new_words.pop(i + 1)  # 뒤 단어 삭제

        # 5. 다음 단어로 이동
        i += 1

    return " ".join(new_words)

# Step 4: 데이터프레임에 적용
df1["doc_split_filtered_1"] = df1["doc_split_filtered"].apply(remove_adjacent_words_with_bigram)
df1

,doc_id,doc_raw,doc_split_updated,r_no,ho_no,doc_split_filtered,doc_split_filtered_1
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 강자_약자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리_소리 소리...,1,1,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 다수_인민 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 다...,1,1,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 세계_사람 사람 야수 사람 사람_진화 진화 진화_진화 진화 지식 진화 도덕 동...,2,1,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 사람_세계 세계 세계_사람 사람 사람_세계 세계 세계_세계 세계 세계_사람 사...,2,1,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하_천하 천하 理解 오...,2,1,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...
...,...,...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 교과_書 書 실상 미국 富豪 캐나다 영국 영국_미국 미국...,334,72,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제 국제_연맹 연맹 연맹_국제 국제 국제_연맹 연맹 원인 경제 機械 ...,334,72,캐나다 미국 국제 국제_연맹 연맹 국제 국제_연맹 연맹 원인 경제 機械 캐나다 산업...,캐나다 미국 국제_연맹 국제_연맹 원인 경제 機械 캐나다 산업 북미 자본 점령 부분...
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 영국_미국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본...,334,72,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...


In [ ]:
# 언더바가 포함된 단어의 빈도 계산
bigram_frequencies = count_underscored_bigrams(df1["doc_split_filtered_1"])

# 빈도를 데이터프레임으로 변환 및 정렬
bigram_freq_df1 = pd.DataFrame(bigram_frequencies.items(), columns=["bigram", "frequency"])
bigram_freq_df1 = bigram_freq_df1.sort_values(by="frequency", ascending=False).reset_index(drop=True)

# 결과 출력
bigram_freq_df1

,bigram,frequency
0,朝鮮_人,981
1,노동_者,489
2,사회_主義,432
3,자본_主義,415
4,소작_人,380
...,...,...
188,私有_재산,30
189,사회_사상,30
190,청년_會,30
191,국제_연맹,30


In [ ]:
print(bigram_freq_df1['bigram'].tolist())

['朝鮮_人', '노동_者', '사회_主義', '자본_主義', '소작_人', '朝鮮_민족', '朝鮮_사람', '主義_者', '자본_家', '무산_계급', '사람_性', '사회_운동', '기독_敎', '朝鮮_사회', '帝國_主義', '中産_계급', '천도_敎', '無_정부', '농업_노동', '정부_主義', '사회_생활', '단체_생활', '일본_人', '계급_意識', '식민_地', '지식_계급', '노동_계급', '사회_문제', '민족_主義', '노동_문제', '인류_사회', '민족_개조', '사회_제도', '공산_主義', '무산_者', '문화_운동', '指導_者', '사람_自己', '일반_민중', '사회_조직', '인생_觀', '민족_생활', '공산_黨', '민주_主義', '문제_해결', '외국_人', '생산_力', '민족_감정', '일반_사회', '자유_평등', '어린_이', '마르크스_主義', '민족_性', '지주_소작', '자본_계급', '新_문화', '계급_투쟁', '사회_현상', '현대_문명', '정치_경제', '생활_難', '중심_인물', '지배_계급', '생활_조건', '현대_사회', '인류_생활', '사상_家', '경제_정책', '人道_正義', '朝鮮_민중', '자연_主義', '농촌_문제', '노동_운동', '지배_者', '도착_點', '현대_人', '약소_민족', '專門_家', '생활_표준', '물산_장려', '프랑스_혁명', '개인_主義', '계급_운동', '유산_者', '자유_主義', '정치_운동', '당국_者', '사상_문화', '정치_家', '중심_세력', '문화_생활', '태평_洋', '新_사회', '종교_家', '唯物_論', '不_합리', '유럽_대전', '하나_님', '正_반대', '출발_點', '不_완전', '서양_人', '생활_費', '사회_계급', '인간_생활', '사회_진화', '소작_제도', '민족_체면', '사상_혁명', '朝鮮_운동', '민족_운동', '중류_계급', '경제_문제', '생활_양식', '인류_구제', '

In [ ]:
df2 = df1[['doc_id', 'doc_raw', 'doc_split_filtered_1', 'r_no', 'ho_no']]
df3 = df2.rename(columns = {'doc_split_filtered_1':'doc_split_12gram'})
df3

,doc_id,doc_raw,doc_split_12gram,r_no,ho_no
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...,1,1
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...,1,1
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...,2,1
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...,2,1
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...,2,1
...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,72
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...,334,72
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제_연맹 국제_연맹 원인 경제 機械 캐나다 산업 북미 자본 점령 부분...,334,72
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...,334,72


## 2.3.구간 및 필자 정보 결합

In [ ]:
def combine_to_basic_df(df, r, ho, writer):
    '''
    # 개벽 데이터 정보 결합 -> 분석의 기본 df 생성
    '''
    df_r = pd.merge(df, r, left_on = 'r_no', right_on = 'r_id', how = 'inner')
    df_rho = pd.merge(df_r, ho, left_on = 'ho_no', right_on = 'ho_id', how = 'inner')
    df_rho_writer = pd.merge(df_rho, writer, left_on = 'w_new', right_on = 'w_new_id', how = 'inner')
    return_df = df_rho_writer[['doc_id', 'doc_raw', 'doc_split_12gram',  'r_no',  'title',  'w_new',  'ho_no',  'grid_1', 'wn_cls']]
    return return_df

In [ ]:
# 기본 데이터프레임 생성 ('1.개요'의 변수를 불러와서 사용)
gb_df = combine_to_basic_df(df3, r, ho, writer)
print(gb_df.shape)
gb_df

(6802, 9)


,doc_id,doc_raw,doc_split_12gram,r_no,title,w_new,ho_no,grid_1,wn_cls
0,1,創刊辭 强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다 東西南北...,창간 辭 강자 약자 優者 劣者 동서 남북 사해 팔방 소리 소리 판단 좌우 間 다수 ...,1,創刊辭,uk01,1,01q,0
1,2,哲人은 말하되 多數 人民의 聲은 곳 神의 聲이라 하엿나니 神은 스스로 要求가 없는지...,哲人 다수 인민 요구 인민 소리 요구 발표 갈앙 인민 소리 갈앙 다수 인민 갈앙 요...,1,創刊辭,uk01,1,01q,0
2,3,世界를 알라 사람은 天使도 안이며 野獸도 안이오 오즉 사람일 뿐이로다 이만치 進化된...,세계 사람 야수 사람 진화 진화 지식 진화 도덕 동물 세계 천당 지옥 세계 진화 국...,2,世界를 알라,uk01,1,01q,0
3,4,사람과 世界는 決코 논하볼 것이 안이엇다 사람으로 된 世界 世界로 된 사람 둘이 안...,사람 세계 사람 세계 세계 사람 세계 대표 시대 가치 사람 대표 문화 상징 符號 사...,2,世界를 알라,uk01,1,01q,0
4,5,過去는 論할 것이 업도다만은 今日과 가티 交通이 이마마하고 知識이 이마마하고 一切의...,過去 금일 교통 지식 일체 문물 오늘 세계 理解 공자 천하 천하 理解 오늘 날 사람...,2,世界를 알라,uk01,1,01q,0
...,...,...,...,...,...,...,...,...,...
6797,6798,미국에서는 每人에게 奴僕이 식 돌아감니다 단 機械奴僕만 이건 무슨 말이냐 하면 기계...,미국 노복 機械 노복 機械 力 운전 계산 人力 계산 미국 국민 사람 機械 輸入 숫자...,334,유로빠와 아메리카(一) 금년 봄에 모쓰크바 엑쓰페리멘탈 劇場에서 한 「트로츠끼」의 講演.,김철산,72,24q,2
6798,6799,매년 저축은ㅡ필요의 비용을 다 쓴 뒤의 것 말이지요ㅡ매년에 억 딸라 金 루불로는 억...,저축 필요 비용 미국 교과 書 실상 미국 富豪 캐나다 영국 미국 부분 미국 캐나다 ...,334,유로빠와 아메리카(一) 금년 봄에 모쓰크바 엑쓰페리멘탈 劇場에서 한 「트로츠끼」의 講演.,김철산,72,24q,2
6799,6800,그러고 또 캐나다는 아조 겸손스럽게 좀 유순하게 미국의 北邊繼續이라고ㅡ국제연맹의 축...,캐나다 미국 국제_연맹 국제_연맹 원인 경제 機械 캐나다 산업 북미 자본 점령 부분...,334,유로빠와 아메리카(一) 금년 봄에 모쓰크바 엑쓰페리멘탈 劇場에서 한 「트로츠끼」의 講演.,김철산,72,24q,2
6800,6801,25년 전에는 영국이 美보다 배나 더 되게 輸入하엿섯음니다 캐나다 사람들은 지금 그...,영국 미국 輸入 캐나다 사람 영국 一部 미국 호주 캐나다 진화 호주 일본 침입 보호...,334,유로빠와 아메리카(一) 금년 봄에 모쓰크바 엑쓰페리멘탈 劇場에서 한 「트로츠끼」의 講演.,김철산,72,24q,2


In [ ]:
# 새 통합 데이터 저장
gb_df.to_excel(file_path + 'result/gb_data_2(doc,1g2g,wn_cls,20250414).xlsx', index=False)

# The End of Note